In [19]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from collections import OrderedDict


## 读入(这段读的是aigame作业的下发json)

In [28]:
all_json = []

aigame_botid = ["6048fc6b81fb3b738e911e3b",
               "6048fcf381fb3b738e912cb8",
               "6048fd3781fb3b738e9138ac",
               "6048fd7981fb3b738e9140fc",
               "6048fda981fb3b738e914488"]

def get_all_json(cwd):
    cur_dir = os.listdir(cwd)  
    for i in cur_dir:
        sub_dir = os.path.join(cwd, i)
        if os.path.isdir(sub_dir):
            get_all_json(sub_dir)
        else:
            if i[-5:] == ".json":
                all_json.append(cwd + "/" + i)
                
get_all_json("../data")

## 读入(这段读的是botzone上下载的对局数据，nb_bots是天梯上排名前30的botid)

In [33]:
all_matches = []

def get_all_matches(cwd):
    cur_dir = os.listdir(cwd)  
    for i in cur_dir:
        sub_dir = os.path.join(cwd, i)  
        if os.path.isdir(sub_dir):
            get_all_matches(sub_dir)
        else:
            if i[-8:] == ".matches":
                all_matches.append(cwd + "/" + i)

get_all_matches("../data")
nb_bots = open("nbbot.txt", "r").read().split('\n')

## 预处理 Combo

In [34]:
def getCardId(card):
    # 求一张牌的 id
    if card < 52:
        return card // 4
    else:
        return card - 39

def getCombo(cards):
    # a combo is represented as a tuple(k, l, r, w)
    # 表示有 k * [l, r] 即 k 张 [l, r] 中的牌（作为主体）w \in [0, 1, 2] 表示带的是啥类型
    if len(cards) == 0:
        return (0, 0, 0, 0)
    tmp = np.zeros(15, dtype = int)
    for card in cards:
        tmp[getCardId(card)] += 1
    k = np.max(tmp)
    l = np.min(np.where(tmp == k))
    r = np.max(np.where(tmp == k))
    w = 0
    if k == 3:
        w = len(cards) // (r - l + 1) - 3
    if k == 4:
        w = (len(cards) // (r - l + 1) - 4) // 2
    return (k, l, r, w)

combo_dict = {}
combo_list = []
combo_cnt = 0

def initCombo():
    global combo_dict, combo_list, combo_cnt
    combo_dict = {}
    combo_list = []
    combo_cnt = 0
    def addCombo(combo):
        global combo_dict, combo_list, combo_cnt
        combo_list.append(combo)
        combo_dict[combo] = combo_cnt
        combo_cnt += 1

    minLength = [0, 5, 3, 2, 2]
    maxWings = [0, 1, 1, 3, 3]
    fold = [0, 0, 0, 1, 2]
    for k in range(1, 5):
        for x in range(13):
            for w in range(maxWings[k]):
                addCombo((k, x, x, w))
        for l in range(12):
            for r in range(l + minLength[k] - 1, 12):
                for w in range(maxWings[k]):
                    if (r - l + 1) * (k + w * fold[k]) <= 20:
                        addCombo((k, l, r, w))
    addCombo((1, 13, 13, 0))
    addCombo((1, 14, 14, 0))
    addCombo((1, 13, 14, 0))
    addCombo((0, 0, 0, 0))
    
initCombo()

def getPartition(cards):
    # 把一次出牌的编号集合划分成 mainbody 和 bywings
    # 其中 mainbody 是一个 list ，bywings 中每个 wing 是一个 list ，也就是一个 list 的 list
    combo = getCombo(cards)
    tmp = [[] for i in range(15)]
    for card in cards:
        tmp[getCardId(card)].append(card)
    mainbody, bywings = [], []
    for i in range(15):
        if len(tmp[i]) > 0:
            if combo[1] <= i and i <= combo[2]:
                mainbody.extend(tmp[i])
            else:
                bywings.append(tmp[i])
    return mainbody, bywings

def getComboMask(combo):
    # 给出一个 combo ，返回可以接在其后面牌型 mask 
    mask = np.zeros(combo_cnt)
    if combo == (0, 0, 0, 0):
        mask = np.ones(combo_cnt)
        mask[combo_dict[(0, 0, 0, 0)]] = 0
        return mask
    mask[combo_dict[(0, 0, 0, 0)]] = 1

    if combo == (1, 13, 14, 0):
        return mask
    mask[combo_dict[(1, 13, 14, 0)]] = 1

    if combo[0] == 4 and combo[1] == combo[2] and combo[3] == 0:
        for i in range(combo[1] + 1, 13):
            mask[combo_dict[(4, i, i, 0)]] = 1
        return mask
    for i in range(13):
        mask[combo_dict[(4, i, i, 0)]] = 1

    for cb in combo_list:
        if cb[0] == combo[0] and cb[2] - cb[1] == combo[2] - combo[1] and cb[3] == combo[3] and cb[1] > combo[1]:
            mask[combo_dict[cb]] = 1
            
    return mask


In [35]:
class Game(object):
    # 这里 0 始终是地主，1 始终是地主下家，2 始终是地主上家

    def __init__(self, init_data):
        self.hand = [np.zeros(15, dtype = int), np.zeros(15, dtype = int), np.zeros(15, dtype = int)]
        for player in range(3):
            for card in init_data[player]:
                self.hand[player][getCardId(card)] += 1
    
    def play(self, player, cards):
        # 模拟打牌 打出 cards 这个 list 中的所有牌
        for card in cards:
            self.hand[player][getCardId(card)] -= 1
            
    def possess(self, player, combo):
        # 判断 player 这个玩家是否拥有 combo 这个牌型的牌
        if combo == (0, 0, 0, 0):
            return True
        for i in range(combo[1], combo[2] + 1):
            if self.hand[player][i] < combo[0]:
                return False
            
        fold = [0, 0, 0, 1, 2]
        need_wings = (combo[2] - combo[1] + 1) * fold[combo[0]] if combo[3] > 0 else 0
        for i in range(15):
            if i < combo[1] or i > combo[2]:
                if self.hand[player][i] >= combo[3]:
                    need_wings -= 1
        if need_wings > 0:
            return False
        return True
    
    def getPossessMask(self, player):
        # 返回 player 拥有的牌型 mask
        mask = np.zeros(combo_cnt)
        for i in range(combo_cnt):
            if self.possess(player, combo_list[i]) == True:
                mask[i] = 1
        return mask
    
    def getMask1(self, player, combo):
        # getPossessMask 和 getComboMask 取交集
        return self.getPossessMask(player) * getComboMask(combo)
    
    def getMask2(self, player, combo, already_played):
        # 带翼的 mask，哪些翼是可以打的？
        # mask 的大小是 15 或者 13, 表示 15 种单牌和 13 种对子
        # 指明 combo 后：(1)少于1/2张的不能打 (2)和主体部分重复的不能打 (3)打过的不能打
        mask = np.ones(15 if combo[3] == 1 else 13)
        for i in range(mask.shape[0]):
                if self.hand[player][i] < combo[3]:
                    mask[i] = 0
        mask[range(combo[1], combo[2] + 1)] = 0
        mask[already_played] = 0
        return mask
    
    def getInput(self, player):
        # 返回两个网络的输入
        # 这里包含五个部分：我自己每种牌的数量、对手每种牌的数量、我的顺子情况、三个人还剩多少张牌、我拥有牌型的 mask
        # size = 4 * 15 + 4 * 15 + 4 * 12 + 3 * 20 + 379
        p1 = (player + 1) % 3
        p2 = (player + 2) % 3

        myhand = np.zeros((4, 15))
        othershand = np.zeros((4, 15))
        for i in range(4):
            myhand[i, np.where(self.hand[player] == i + 1)] = 1
            othershand[i, np.where(self.hand[p1] + self.hand[p2] == i + 1)] = 1
        
        mystraight = np.zeros((4, 12))
        for i in range(4):
            k = 0
            for j in range(12):
                if self.hand[player][i] >= i + 1:
                    k += 1
                else:
                    k = 0
                mystraight[i, j] = k
                
        handcnt = np.zeros((3, 20))
        for player in range(3):
            handcnt[player, np.sum(self.hand[player]) - 1] = 1

        return np.concatenate([myhand.flatten(), othershand.flatten(), mystraight.flatten(), handcnt.flatten(), self.getPossessMask(player)])
    
_input_size = 60 + 60 + 48 + 60 + combo_cnt

## 定义数据集类

In [36]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.X = []
        self.M = []
        self.Y = []
    def append(self, x, m, y):
        self.X.append(x)
        self.M.append(m)
        self.Y.append(y)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return (self.X[idx], self.M[idx], self.Y[idx])


In [37]:
DS1 = [MyDataset(), MyDataset(), MyDataset()]
DS2 = [MyDataset(), MyDataset()]
# DS1 是三个玩家分别的出牌数据集，可能处于不同位置会有不同的出牌策略所以分别训练了网络
# DS2 是带牌数据集，分别是单排和顺子，因为数据量比较少而且感觉三个人没什么太大区别就合并到一起了

In [38]:
print_cnt = 0

for file in all_json:
    print(file)
    with open(file, 'r+') as f:
        while True:
            data = f.readline()
            if(len(data) == 0):
                break
            data = json.loads(data)
            initdata = json.loads(data['initdata'])

            nb_bot = [0, 0, 0]
            exists_nb = 0
            for i in range(3):
                if "bot" in data['players'][i].keys() and data['players'][i]['bot'] == aigame_botid[0]:
                    nb_bot[i] = 1
                    exists_nb = 1
            if exists_nb == 0:
                break

            g = Game(initdata['allocation'])
            log = data['log']
            
            las_combo = (0, 0, 0, 0)
            las_play = -1
            
            for i in range(1, len(log), 2):
                player = -1
                for p in range(3):
                    if str(p) in log[i]:
                        player = p
                        break
                
                if log[i][str(p)]['verdict'] != 'OK':
                    print(log[i][str(p)]['verdict'])
                    break

                if las_play == player:
                    las_combo = (0, 0, 0, 0)
                    
                cards = log[i][str(player)]['response']
                cur_combo = getCombo(cards)
                mainbody, bywings = getPartition(cards)
                    
                if nb_bot[player]:
                    input_x = g.getInput(player)
                    input_m = g.getMask1(player, las_combo)
                    output_y = combo_dict[cur_combo]
                    assert input_m[output_y] == 1
                    if np.sum(input_m) > 1:
                        DS1[player].append(input_x, input_m, output_y)
                
                g.play(player, mainbody)
                
                if len(bywings) != 0:
                    already_played = []
                    for w in bywings:
                        assert len(w) == 1 or len(w) == 2
                        if nb_bot[player]:
                            input_x = g.getInput(player)
                            input_m = g.getMask2(player, cur_combo, already_played)
                            output_y = getCardId(w[0])
                            assert input_m[output_y] == 1
                            DS2[0 if len(w) == 1 else 1].append(input_x, input_m, output_y)
                            
                        g.play(player, w)
                        already_played.append(getCardId(w[0]))
                
                if cur_combo != (0, 0, 0, 0):
                    las_combo = cur_combo
                    las_play = player
                    
                
    
    print_cnt += 1
    print("files = %d, lengths = (%d, %d, %d, %d, %d)"
          % (print_cnt, len(DS1[0]), len(DS1[1]), len(DS1[2]), len(DS2[0]), len(DS2[1])))


../data/download_bot_matches/1_6048fc6b81fb3b738e911e3b/train/log_1_20.json
files = 1, lengths = (1169, 1007, 1146, 353, 38)
../data/download_bot_matches/1_6048fc6b81fb3b738e911e3b/train/log_1_16.json
files = 2, lengths = (2152, 1807, 2007, 624, 81)
../data/download_bot_matches/1_6048fc6b81fb3b738e911e3b/train/log_1_10.json
files = 3, lengths = (2840, 2408, 2573, 819, 109)
../data/download_bot_matches/1_6048fc6b81fb3b738e911e3b/train/log_1_3.json
files = 4, lengths = (3087, 2607, 2796, 882, 118)
../data/download_bot_matches/1_6048fc6b81fb3b738e911e3b/train/log_1_4.json
files = 5, lengths = (3697, 3108, 3290, 1036, 149)
../data/download_bot_matches/1_6048fc6b81fb3b738e911e3b/train/log_1_5.json
files = 6, lengths = (4058, 3396, 3581, 1145, 163)
../data/download_bot_matches/1_6048fc6b81fb3b738e911e3b/train/log_1_22.json
files = 7, lengths = (5064, 4317, 4569, 1442, 200)
../data/download_bot_matches/1_6048fc6b81fb3b738e911e3b/train/log_1_21.json
files = 8, lengths = (6193, 5322, 5638, 1790

In [39]:
print(len(DS1[0]), len(DS1[1]), len(DS1[2]), len(DS2[0]), len(DS2[1]))

23785 20027 21129 6697 957


## 网络框架

In [40]:
HIDDEN_SIZE = 512
torch.set_default_tensor_type(torch.DoubleTensor)

class MyModule(nn.Module):
    def __init__(self, INPUT_SIZE, OUTPUT_SIZE):
        super(MyModule, self).__init__()
        self.fc = nn.Sequential(OrderedDict([
                ('fc1', nn.Linear(INPUT_SIZE, HIDDEN_SIZE)),
                ('relu', nn.ReLU()),
                ('dropout', nn.Dropout(p = 0.2)),
                ('fc2', nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE))
            ]))
        
    def forward(self, x, m):
        return nn.LogSoftmax(dim = -1)(self.fc(x) * m)
    

## 模型训练（主体）

In [41]:
best_acc = 0

def getPred(output):
    return output.detach().numpy().argmax(axis = 1)

def train(net, data_loader, data_size, criterion, optimizer):
    net.train()
    for i, (x, m, y) in enumerate(data_loader):
        optimizer.zero_grad()
        output = net(x, m)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        
    total_correct = 0
    avg_loss = 0.0
    for i, (x, m, y) in enumerate(data_loader):
        output = net(x, m)
        avg_loss += criterion(output, y).sum()
        pred = getPred(output)
        total_correct += (pred == y.detach().numpy()).sum()
    avg_loss /= data_size
    cur_acc = float(total_correct) / data_size
    print('Training Avg. Loss: %f, Accuracy: %f' % (avg_loss, cur_acc))

def validate(net, data_loader, data_size, criterion, model_name):
    global best_acc
    net.eval()
    total_correct = 0
    avg_loss = 0.0
    for i, (x, m, y) in enumerate(data_loader):
        output = net(x, m)
        avg_loss += criterion(output, y).sum()
        pred = getPred(output)
        total_correct += (pred == y.detach().numpy()).sum()
    
    avg_loss /= data_size
    cur_acc = float(total_correct) / data_size
    print('Validation Avg. Loss: %f, Accuracy: %f' % (avg_loss, cur_acc))
    
    if cur_acc > best_acc:
        best_acc = cur_acc
        torch.save(net.state_dict(), './model/best_model_for_' + model_name + '.pt')


In [42]:
for i in range(3):
        
    print("-----------------------------------------")
    print("Training model %d" % i)
    print("-----------------------------------------")

    net = MyModule(_input_size, combo_cnt)
    best_acc = 0
    
    train_size = int(0.9 * len(DS1[i]))
    valid_size = len(DS1[i]) - train_size
    print("train_size = ", train_size, "valid_size = ", valid_size)


    train_data, valid_data = torch.utils.data.random_split(DS1[i], [train_size, valid_size])

    train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = 128)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = 128)

    for epoch in range(40):
        print("epoch = ", epoch)
        train(net, data_loader = train_loader,
              data_size = train_size,
              criterion = nn.CrossEntropyLoss(),
              optimizer = optim.Adam(net.parameters(), lr = 2e-3))

        validate(net, data_loader = valid_loader,
                 data_size = valid_size,
                 criterion = nn.CrossEntropyLoss(),
                 model_name = str(i) + "mainbody")

-----------------------------------------
Training model 0
-----------------------------------------
train_size =  21406 valid_size =  2379
epoch =  0


/home/zhoushuyu/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


Training Avg. Loss: 0.007699, Accuracy: 0.688358
Validation Avg. Loss: 0.008058, Accuracy: 0.689365
epoch =  1
Training Avg. Loss: 0.006337, Accuracy: 0.751191
Validation Avg. Loss: 0.007214, Accuracy: 0.731400
epoch =  2
Training Avg. Loss: 0.005375, Accuracy: 0.789919
Validation Avg. Loss: 0.006557, Accuracy: 0.748634
epoch =  3
Training Avg. Loss: 0.004711, Accuracy: 0.817481
Validation Avg. Loss: 0.006426, Accuracy: 0.752837
epoch =  4
Training Avg. Loss: 0.004128, Accuracy: 0.841353
Validation Avg. Loss: 0.006198, Accuracy: 0.769651
epoch =  5
Training Avg. Loss: 0.003726, Accuracy: 0.853592
Validation Avg. Loss: 0.006249, Accuracy: 0.770912
epoch =  6
Training Avg. Loss: 0.003356, Accuracy: 0.867467
Validation Avg. Loss: 0.006398, Accuracy: 0.756620
epoch =  7
Training Avg. Loss: 0.003246, Accuracy: 0.868074
Validation Avg. Loss: 0.006473, Accuracy: 0.755780
epoch =  8
Training Avg. Loss: 0.002811, Accuracy: 0.887368
Validation Avg. Loss: 0.006584, Accuracy: 0.759983
epoch =  9
T

Training Avg. Loss: 0.000517, Accuracy: 0.978972
Validation Avg. Loss: 0.012843, Accuracy: 0.747878
epoch =  34
Training Avg. Loss: 0.000580, Accuracy: 0.973924
Validation Avg. Loss: 0.013254, Accuracy: 0.744883
epoch =  35
Training Avg. Loss: 0.000468, Accuracy: 0.980914
Validation Avg. Loss: 0.013110, Accuracy: 0.746380
epoch =  36
Training Avg. Loss: 0.000586, Accuracy: 0.973091
Validation Avg. Loss: 0.014010, Accuracy: 0.725412
epoch =  37
Training Avg. Loss: 0.000459, Accuracy: 0.981081
Validation Avg. Loss: 0.014107, Accuracy: 0.736895
epoch =  38
Training Avg. Loss: 0.000459, Accuracy: 0.980748
Validation Avg. Loss: 0.013896, Accuracy: 0.740389
epoch =  39
Training Avg. Loss: 0.000484, Accuracy: 0.978972
Validation Avg. Loss: 0.014539, Accuracy: 0.739890
-----------------------------------------
Training model 2
-----------------------------------------
train_size =  19016 valid_size =  2113
epoch =  0
Training Avg. Loss: 0.007549, Accuracy: 0.635886
Validation Avg. Loss: 0.0082

## 模型训练（带翼）

In [43]:
for i in range(2):
    
    print("-----------------------------------------")
    print("Training model %d" % i)
    print("-----------------------------------------")
    
    net = MyModule(_input_size, 15 if i == 0 else 13)
    best_acc = 0

    train_size = int(0.9 * len(DS2[i]))
    valid_size = len(DS2[i]) - train_size
    print("train_size = ", train_size, "valid_size = ", valid_size)

    train_data, valid_data = torch.utils.data.random_split(DS2[i], [train_size, valid_size])

    train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = 32)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = 32)

    for epoch in range(40):
        print("epoch = ", epoch)
        train(net, data_loader = train_loader,
               data_size = train_size,
               criterion = nn.CrossEntropyLoss(),
               optimizer = optim.Adam(net.parameters(), lr = 2e-3))
        validate(net, data_loader = valid_loader,
                  data_size = valid_size,
                  criterion = nn.CrossEntropyLoss(),
                  model_name = str(i) + "bywings")

train_size =  6027 valid_size =  670
epoch =  0
Training Avg. Loss: 0.018890, Accuracy: 0.834080
Validation Avg. Loss: 0.021725, Accuracy: 0.822388
epoch =  1
Training Avg. Loss: 0.013937, Accuracy: 0.886013
Validation Avg. Loss: 0.018796, Accuracy: 0.862687
epoch =  2
Training Avg. Loss: 0.012190, Accuracy: 0.894973
Validation Avg. Loss: 0.018690, Accuracy: 0.865672
epoch =  3
Training Avg. Loss: 0.009497, Accuracy: 0.919363
Validation Avg. Loss: 0.018669, Accuracy: 0.877612
epoch =  4
Training Avg. Loss: 0.008127, Accuracy: 0.930977
Validation Avg. Loss: 0.019045, Accuracy: 0.871642
epoch =  5
Training Avg. Loss: 0.007256, Accuracy: 0.931309
Validation Avg. Loss: 0.019508, Accuracy: 0.864179
epoch =  6
Training Avg. Loss: 0.006302, Accuracy: 0.940767
Validation Avg. Loss: 0.019603, Accuracy: 0.868657
epoch =  7
Training Avg. Loss: 0.005071, Accuracy: 0.951220
Validation Avg. Loss: 0.020240, Accuracy: 0.868657
epoch =  8
Training Avg. Loss: 0.005352, Accuracy: 0.946906
Validation Avg.

Training Avg. Loss: 0.000923, Accuracy: 0.994193
Validation Avg. Loss: 0.078191, Accuracy: 0.770833
epoch =  34
Training Avg. Loss: 0.000465, Accuracy: 0.994193
Validation Avg. Loss: 0.079304, Accuracy: 0.770833
epoch =  35
Training Avg. Loss: 0.000409, Accuracy: 0.994193
Validation Avg. Loss: 0.093941, Accuracy: 0.802083
epoch =  36
Training Avg. Loss: 0.000034, Accuracy: 1.000000
Validation Avg. Loss: 0.084536, Accuracy: 0.770833
epoch =  37
Training Avg. Loss: 0.000222, Accuracy: 0.998839
Validation Avg. Loss: 0.081941, Accuracy: 0.770833
epoch =  38
Training Avg. Loss: 0.000295, Accuracy: 0.995354
Validation Avg. Loss: 0.088040, Accuracy: 0.812500
epoch =  39
Training Avg. Loss: 0.000788, Accuracy: 0.993031
Validation Avg. Loss: 0.081396, Accuracy: 0.802083


## main.py

In [44]:
os.environ.get("USER", "") == "root"

False

In [53]:
_BOTZONE_ONLINE = os.environ.get("USER", "") == "root"

my_hand = []
g = Game([[], [], []])
my_pos = -1
others = []
las_combo = (0, 0, 0, 0)

def BIDDING():
    bid_val = 0
    print(json.dumps({
        "response": bid_val
    }))
    if not _BOTZONE_ONLINE:
        assert(0)
    exit()

model_path = "./data/fightlandlord_model/" if _BOTZONE_ONLINE else "./model/"

def PLAYING():
    def getFromHand(idx):
        global my_hand
        for c in my_hand:
            if getCardId(c) == idx:
                my_hand.remove(c)
                return c
    
    to_play = []
    
    model_name = "best_model_for_" + str(my_pos) + "mainbody.pt"
    model = MyModule(_input_size, combo_cnt)
    model.load_state_dict(torch.load(model_path + model_name))
    
    mask = g.getMask1(my_pos, las_combo)
    combo_id = -1
    if np.sum(mask) == 1:
        combo_id = np.argmax(mask)
    else:
        combo_id = model(torch.from_numpy(g.getInput(my_pos)).unsqueeze(0),
                         torch.from_numpy(mask)).unsqueeze(0).detach().numpy().argmax()
    
    combo = combo_list[combo_id]
    for i in range(combo[1], combo[2] + 1):
        for j in range(combo[0]):
            to_play.append(getFromHand(i))
    g.play(my_pos, to_play)
    
    if combo[3] != 0:
        model_name = "best_model_for_" + str(combo[3] - 1) + "bywings.pt"
        model = MyModule(_input_size, (15 if combo[3] == 1 else 13))
        model.load_state_dict(torch.load(model_path + model_name))

        cnt = (combo[2] - combo[1] + 1) * (1 if combo[0] == 3 else 2)
        already_played = []
        for i in range(cnt):
            wing_id = model(torch.from_numpy(g.getInput(my_pos)),
                            torch.from_numpy(g.getMask2(my_pos, combo, already_played))).detach().numpy().argmax()
            tmp = []
            if wing_id < 15:
                tmp = [getFromHand(wing_id)]
            else:
                wing_id -= 15
                tmp = [getFromHand(wing_id), getFromHand(wing_id)]
            g.play(my_pos, tmp)
            to_play.extend(tmp)
            already_played.append(wing_id)
    
    print(json.dumps({
        "response": to_play
    }))
    if not _BOTZONE_ONLINE:
        assert 0
    exit()
    
if __name__ == "__main__":
    initCombo()
    data = json.loads(input())
    my_hand, others_hand = data["requests"][0]["own"], []
    for i in range(54):
        if i not in my_hand:
            others_hand.append(i)

    TODO = "bidding"
    if "bid" in data["requests"][0]:
        bid_list = data["requests"][0]["bid"]
    
    for i in range(len(data["requests"])):
        request = data["requests"][i]
        
        if "publiccard" in request:
            bot_pos = request["pos"]
            lord_pos = request["landlord"]
            my_pos = (bot_pos - lord_pos + 3) % 3
            others = [(my_pos + 1) % 3, (my_pos + 2) % 3]
            tmp = [[], [], []]
            if my_pos == 0:
                my_hand.extend(request["publiccard"])
                tmp[0] = my_hand
                tmp[1], tmp[2] = others_hand[:17], others_hand[17:] # 随便分
            else:
                tmp[my_pos] = my_hand
                tmp[0] = others_hand[:20]
                tmp[2 if my_pos == 1 else 1] = others_hand[20:]
            g = Game(tmp)
            
        if "history" in request:
            history = request["history"]
            TODO = "playing"
            for j in range(2):
                p = others[j]
                cards = history[j]
                g.play(p, cards)
                cur_combo = getCombo(cards)
                if cur_combo != (0, 0, 0, 0):
                    las_combo = cur_combo

            if i < len(data["requests"]) - 1:
                cards = data["responses"][i]
                g.play(my_pos, cards)
                for c in cards:
                    my_hand.remove(c)
                las_combo = (0, 0, 0, 0)
    
    
    if TODO == "bidding":
        BIDDING()
    else:
        PLAYING()

{"requests":[{"own":[34,48,26,6,46,37,51,36,1,27,17,5,24,29,18,52,53],"bid":[0,0]},{"history":[[7,9,10,11],[4,32,33,35]],"own":[34,48,26,6,46,37,51,36,1,27,17,5,24,29,18,52,53],"publiccard":[39,0,9],"landlord":0,"pos":2,"finalbid":1},{"history":[[0,44,45,47],[]]},{"history":[[],[]]}],"responses":[0,[],[52,53]]}
{"response": [26, 27, 24, 1]}


AssertionError: 